In [3]:
from sknetwork.ranking import PageRank
import pickle
from causallearn.search.FCMBased import lingam
from causallearn.utils.cit import chisq, fisherz, gsq, kci, mv_fisherz
import networkx as nx
from collections import defaultdict
import pandas as pd
import time
import networkx as nx
import random
from collections import defaultdict
import numpy as np
from sklearn.preprocessing import normalize

In [4]:
class DiWalk:
    def __init__(self, graph, start_node, rho=0.5):
        """
        Initialize AutoMAP with graph GA, starting node, and rho parameter.
        
        :param graph: networkx.DiGraph representing the service-calling topology
        :param start_node: Node from which the random walk starts
        :param rho: Parameter controlling backward transition strength (0 <= rho < 1)
        """
        self.GA = graph
        if start_node not in self.GA:
            raise ValueError(f"Start node '{start_node}' is not in the graph.")
        self.start_node = start_node
        if not (0 <= rho < 1):
            raise ValueError("Parameter rho must be in the range [0, 1).")
        self.rho = rho
        self.visit_counts = defaultdict(int)
    
    def random_walk(self, steps=100, seed=None):
        """
        Perform random walk for a given number of steps.
        
        :param steps: Number of steps to simulate
        :param seed: Random seed for reproducibility
        """
        if seed is not None:
            random.seed(seed)
        
        current_node = self.start_node
        self.visit_counts[current_node] += 1
        
        for step in range(steps):
            Oi = list(self.GA.successors(current_node))
            Ii = list(self.GA.predecessors(current_node))
            
            # Assign weights
            forward_weight = len(Oi)  # Each forward transition has weight=1
            backward_weight = len(Ii) * self.rho  # Each backward transition has weight=rho
            self_weight = 1  # Self-transition weight=1
            
            total_weight = forward_weight + backward_weight + self_weight
            if total_weight == 0:
                # If no transitions are possible, stay on the current node
                self.visit_counts[current_node] += 1
                continue
            
            probabilities = []
            nodes = []
            
            # Forward transitions
            for neighbor in Oi:
                probabilities.append(1 / total_weight)
                nodes.append(neighbor)
            
            # Backward transitions
            for neighbor in Ii:
                probabilities.append(self.rho / total_weight)
                nodes.append(neighbor)
            
            # Self-transition
            probabilities.append(self_weight / total_weight)
            nodes.append(current_node)
            
            # Choose next node based on probabilities
            next_node = random.choices(nodes, weights=probabilities, k=1)[0]
            self.visit_counts[next_node] += 1
            current_node = next_node
    
    def get_sorted_visit_counts(self):
        """
        Get the sorted list of nodes based on visit counts.
        
        :return: List of tuples sorted by visit counts in descending order
        """
        sorted_counts = sorted(self.visit_counts.items(), key=lambda x: x[1], reverse=True)
        return sorted_counts

    def reset_counts(self):
        """
        Reset the visit counts.
        """
        self.visit_counts = defaultdict(int)

In [5]:
hts_res_dict = {}
for experiment in range(0,4):
    with open(f"../data/fault_data/hts_fault_{experiment}.pkl", "rb") as f:
        s_list, X = pickle.load(f)
    ind = 0
    mapping = {}
    for service in s_list:
        x = service.split('-')
        x = x[:-2]
        x = '-'.join(x)
        mapping[ind] = x
        ind+=1
    # Start CausalRCA
    X = np.diff(X,axis=0)
    X = normalize(X,axis=0)
    start_time = time.time()
    model = lingam.DirectLiNGAM()
    model.fit(X)
    adj = model.adjacency_matrix_.T 
    G = nx.from_numpy_array(adj, create_using=nx.DiGraph)
    G = G.reverse(copy=True)
    total_visit_counts = defaultdict(int)
    steps_per_walk = 10
    for i in range(46):
        for rep in range(0,100):
            walker = DiWalk(graph=G, start_node=i, rho=0)
            walker.random_walk(steps=steps_per_walk)
            
            # Accumulate the visit counts from this walk
            for node, count in walker.visit_counts.items():
                total_visit_counts[node] += count
    
    total_f = total_visit_counts
    combined_frequencies = defaultdict(float)
    
    i = 0
    mapping = {}
    for service in s_list:
        x = service.split('-')
        x = x[:-2]
        x = '-'.join(x)
        mapping[i] = x
        i+=1
        
    for service_id, frequency in total_f.items():
        service_type = mapping.get(service_id, 'unknown')
        combined_frequencies[service_type] += frequency
    
    # Convert the combined frequencies dictionary to a pandas dataframe
    combined_frequencies_df = pd.DataFrame(
        list(combined_frequencies.items()), columns=["Service Type", "Combined Frequency"]
    )
    end_time = time.time()
    # ####
    # 
    t_elapsed = end_time - start_time
    sorted_pr = combined_frequencies_df.sort_values(by="Combined Frequency", ascending=False)
    # top_5 = list(sorted_pr.head(5)["Service Type"])
    hts_res_dict[experiment] = (sorted_pr, t_elapsed)

In [6]:
cp_res_dict = {}
for experiment in range(0,4):
    with open(f"../data/fault_data/cp_fault_{experiment}.pkl", "rb") as f:
        s_list, X = pickle.load(f)
    ind = 0
    mapping = {}
    for service in s_list:
        x = service.split('-')
        x = x[:-2]
        x = '-'.join(x)
        mapping[ind] = x
        ind+=1
    # Start CausalRCA
    X = np.diff(X,axis=0)
    X = normalize(X,axis=0)
    start_time = time.time()
    model = lingam.DirectLiNGAM()
    model.fit(X)
    adj = model.adjacency_matrix_.T 
    G = nx.from_numpy_array(adj, create_using=nx.DiGraph)
    G = G.reverse(copy=True)
    total_visit_counts = defaultdict(int)
    steps_per_walk = 10
    for i in range(46):
        for rep in range(0,100):
            walker = DiWalk(graph=G, start_node=i, rho=0.1)
            walker.random_walk(steps=steps_per_walk)
            
            # Accumulate the visit counts from this walk
            for node, count in walker.visit_counts.items():
                total_visit_counts[node] += count
    
    total_f = total_visit_counts
    combined_frequencies = defaultdict(float)
    
    i = 0
    mapping = {}
    for service in s_list:
        x = service.split('-')
        x = x[:-2]
        x = '-'.join(x)
        mapping[i] = x
        i+=1
        
    for service_id, frequency in total_f.items():
        service_type = mapping.get(service_id, 'unknown')
        combined_frequencies[service_type] += frequency
    
    # Convert the combined frequencies dictionary to a pandas dataframe
    combined_frequencies_df = pd.DataFrame(
        list(combined_frequencies.items()), columns=["Service Type", "Combined Frequency"]
    )
    end_time = time.time()
    # ####
    # 
    t_elapsed = end_time - start_time
    sorted_pr = combined_frequencies_df.sort_values(by="Combined Frequency", ascending=False)
    # top_5 = list(sorted_pr.head(5)["Service Type"])
    cp_res_dict[experiment] = (sorted_pr, t_elapsed)